# Data here: 
## One and two day type 
## Three day type

# Day Type: One/two
## Create Date and Day of Week columns in the DF

In [1]:
#importing packages needed for analysis
import os
import numpy as np
import pandas as pd
import math
from pandas import DataFrame

path = os.getcwd()
#print(path)

#this code creates an output directory in the parent director, if one does not exist yet
#Note: this is where all of the output files will be written, since outputs are large this saves space in git
path = os.getcwd()
parent = os.path.dirname(path)
outputs_dir = parent+'\outputs'
if not os.path.exists(outputs_dir):
    os.makedirs(outputs_dir)
print('output files are written out in parent directory: '+outputs_dir)

load_dur = pd.read_csv('../outputs/load_long_format.csv')
solar_dur = pd.read_csv('../outputs/solar_long_format.csv')
wind_dur = pd.read_csv('../outputs/wind_long_format.csv')

## UNCOMMENT WHICH PROFILE TO BE USED
#x = load_dur
#x_name = 'load'
#x_name2 = 'Load'
#x_column = 'Load'

#x = solar_dur
#x_name = 'solar'
#x_name2 = 'Solar_Gen'
#choose TRG 
#x_column = 'TRG6'

x = wind_dur
x_name = 'wind'
x_name2 = 'Wind_Gen'
x_column = 'TRG4'

x = x[['Region','R_Group','R_Subgroup','Season','Month','Day','Hour',x_column]]
years = pd.read_csv('inputs/years.csv').dropna()

#print(x)

output files are written out in parent directory: C:\Users\tgoforth\Documents\IPM temporal resolution project\outputs


In [2]:
from itertools import cycle

#repeat the date for 24 times for each hour in the day
year_2016 = years['2016'].repeat(24).reset_index(drop=True)
year_2011 = years['2011'].repeat(24).reset_index(drop=True)
#print(year_2016)

#use itertools.cycle to make the date repeat until it reaches the end of the dataframe
year_2016 = cycle(year_2016)
year_2011 = cycle(year_2011)

#create temporary data frames where they will iterate through. Needed to achieve the correct length of the DF
temp2016 = x.loc[x['R_Group'] == 'ERC'].copy()
temp2011 = x.loc[x['R_Group'] != 'ERC'].copy()

#iterate through the date until the end of DF 
temp2016['Date'] = [next(year_2016) for year in range(len(temp2016))]
temp2011['Date'] = [next(year_2011) for year in range(len(temp2011))]
#print(temp2016)
#print(temp2011)

x2 = pd.concat([temp2016, temp2011], ignore_index=True)
#print(x2)

#convert date to a datetime type 
x2['Date'] = pd.to_datetime(x2['Date'])
x2['DOW'] = x2['Date'].dt.weekday

#check if it is a weekday or not 
weekday = pd.read_csv('inputs/weekday.csv')
x2 = pd.merge(x2,weekday,on='DOW',how='left')
#print(x2)

## Case 1: Monthly, single day type, 24 hours (288 segments)
#### Methodology: Using groupby function to group first by month, then by 24 hours

In [3]:
case1_x = x2.copy()

aggregations = {x_column:['count',sum,'mean']}
case1 = case1_x.groupby(['Region','Month','Hour'],as_index=False).agg(aggregations)
case1.columns = case1.columns.droplevel(0)
case1.columns = ['Region','Month','Hour','Hour_Tot',x_name2+'_Tot',x_name2+'_Avg']
print(case1.head())
print('number of rows in dataset =',case1.shape[0])
case1.to_csv('../outputs/'+x_name+'_segments_2daytype_monthly_24hr.csv')
print()

case1_x2 = pd.merge(case1_x,case1,on=['Region','Month','Hour'],how='left')
case1_x2 = case1_x2.sort_values(['Region',x_column]).drop(case1_x2.columns[0], axis=1)
print(case1_x2.head(3))
print('number of rows in dataset =',case1_x2.shape[0])
case1_x2.to_csv('../outputs/'+x_name+'_8760_2daytype_monthly_24hr.csv')

     Region  Month  Hour  Hour_Tot  Wind_Gen_Tot  Wind_Gen_Avg
0  ERC_PHDL      1     1        31       17461.0    563.258065
1  ERC_PHDL      1     2        31       17975.0    579.838710
2  ERC_PHDL      1     3        31       18119.0    584.483871
3  ERC_PHDL      1     4        31       18400.0    593.548387
4  ERC_PHDL      1     5        31       18581.0    599.387097
number of rows in dataset = 16704

     R_Group R_Subgroup  Season  Month  Day  Hour  TRG4       Date  DOW  \
5913     ERC       PHDL  summer      9  247    10  33.0 2016-09-03    5   
5914     ERC       PHDL  summer      9  247    11  49.0 2016-09-03    5   
4883     ERC       PHDL  summer      7  204    12  51.0 2016-07-22    4   

      Weekday  Hour_Tot  Wind_Gen_Tot  Wind_Gen_Avg  
5913    False        30        8956.0    298.533333  
5914    False        30        8981.0    299.366667  
4883     True        31        6916.0    223.096774  
number of rows in dataset = 867240


## Case 2: Season, single day-type, 24 hours (72 segments)
#### Methodology: Use groupby function to group by season and hour

In [4]:
case2_x = x2.copy()
case2_seasons = pd.read_csv('inputs/season_bimonthly.csv')
case2_seasons = case2_seasons.drop(['bimonthly'], axis=1)
case2_seasons = case2_seasons.rename(columns={'seasonal':'Season_Group','month':'Month'})

case2_x = pd.merge(case2_x, case2_seasons, on='Month', how='left')

aggregations = {x_column:['count',sum,'mean']}
case2 = case2_x.groupby(['Region','Season_Group','Hour'],as_index=False).agg(aggregations)
case2.columns = case2.columns.droplevel(0)
case2.columns = ['Region','Season_Group','Hour','Hour_Tot',x_name2+'_Tot',x_name2+'_Avg']
print(case2.head())
print('number of rows in dataset =',case2.shape[0])
case2.to_csv('../outputs/'+x_name+'_segments_2daytype_season_24hr.csv')
print()

case2_x2 = pd.merge(case2_x,case2,on=['Region','Season_Group','Hour'],how='left')
case2_x2 = case2_x2.sort_values(['Region',x_column]).drop(case2_x2.columns[0],axis=1)
print(case2_x2.head(3))
print('number of rows in dataset =',case2_x2.shape[0])
case2_x2.to_csv('../outputs/'+x_name+'_8760_2daytype_season_24hr.csv')

     Region  Season_Group  Hour  Hour_Tot  Wind_Gen_Tot  Wind_Gen_Avg
0  ERC_PHDL             1     1        90       50327.0    559.188889
1  ERC_PHDL             1     2        90       51509.0    572.322222
2  ERC_PHDL             1     3        90       52196.0    579.955556
3  ERC_PHDL             1     4        90       52972.0    588.577778
4  ERC_PHDL             1     5        90       53882.0    598.688889
number of rows in dataset = 6960

     R_Group R_Subgroup  Season  Month  Day  Hour  TRG4       Date  DOW  \
5913     ERC       PHDL  summer      9  247    10  33.0 2016-09-03    5   
5914     ERC       PHDL  summer      9  247    11  49.0 2016-09-03    5   
4883     ERC       PHDL  summer      7  204    12  51.0 2016-07-22    4   

      Weekday  Season_Group  Hour_Tot  Wind_Gen_Tot  Wind_Gen_Avg  
5913    False             3        61       20009.0    328.016393  
5914    False             3        61       20059.0    328.836066  
4883     True             5        62    

## Case 3: Monthly, weekend/weekday, 24 hours (576 segments)
#### Metholodogy: Use groupby function to group by month, then weekend/weekday, then by 24 hours

In [5]:
case3_x = x2.copy()

aggregations = {x_column:['count',sum,'mean']}
case3 = case3_x.groupby(['Region','Month','Weekday','Hour'],as_index=False).agg(aggregations)
case3.columns = case3.columns.droplevel(0)
case3.columns = ['Region','Month','Weekday','Hour','Hour_Tot',x_name2+'_Tot',x_name2+'_Avg']
print(case1.head())
print('number of rows in dataset =',case3.shape[0])
case3.to_csv('../outputs/'+x_name+'_segments_2daytype_monthly_wkd.csv')
print()

case3_x2 = pd.merge(case3_x,case3,on=['Region','Month','Weekday','Hour'],how='left')
case3_x2 = case3_x2.sort_values(['Region',x_column]).drop(case3_x2.columns[0],axis=1)
print(case3_x2.head())
print('number of rows in dataset =',case3_x2.shape[0])
case3_x2.to_csv('../outputs/'+x_name+'_8760_2daytype_monthly_wkd.csv')

     Region  Month  Hour  Hour_Tot  Wind_Gen_Tot  Wind_Gen_Avg
0  ERC_PHDL      1     1        31       17461.0    563.258065
1  ERC_PHDL      1     2        31       17975.0    579.838710
2  ERC_PHDL      1     3        31       18119.0    584.483871
3  ERC_PHDL      1     4        31       18400.0    593.548387
4  ERC_PHDL      1     5        31       18581.0    599.387097
number of rows in dataset = 29418

     R_Group R_Subgroup  Season  Month  Day  Hour  TRG4       Date  DOW  \
5913     ERC       PHDL  summer      9  247    10  33.0 2016-09-03    5   
5914     ERC       PHDL  summer      9  247    11  49.0 2016-09-03    5   
4883     ERC       PHDL  summer      7  204    12  51.0 2016-07-22    4   
4880     ERC       PHDL  summer      7  204     9  53.0 2016-07-22    4   
5028     ERC       PHDL  summer      7  210    13  55.0 2016-07-28    3   

      Weekday  Hour_Tot  Wind_Gen_Tot  Wind_Gen_Avg  
5913    False         8        2380.0    297.500000  
5914    False         8     

## Case 4: Season, weekend/weekday, 24 hours (144 segments)
#### Methodology: groupby season, weekday, hour

In [6]:
case4_x = x2.copy()
case4_seasons = pd.read_csv('inputs/season_bimonthly.csv')
case4_seasons = case4_seasons.drop(['bimonthly'], axis=1)
case4_seasons = case4_seasons.rename(columns={'seasonal':'Season_Group','month':'Month'})

case4_x = pd.merge(case4_x, case4_seasons, on='Month', how='left')
#print(case4_x)

aggregations = {x_column:['count',sum,'mean']}
case4 = case4_x.groupby(['Region','Season_Group','Weekday','Hour'],as_index=False).agg(aggregations)
case4.columns = case4.columns.droplevel(0)
case4.columns = ['Region','Season_Group','Weekday','Hour','Hour_Tot',x_name2+'_Tot',x_name2+'_Avg']
print(case4.head())
print('number of rows in dataset =',case4.shape[0])
case4.to_csv('../outputs/'+x_name+'_segments_2daytype_season_wkd.csv')
print()

case4_x2 = pd.merge(case4_x,case4,on=['Region','Season_Group','Weekday','Hour'],how='left')
case4_x2 = case4_x2.sort_values(['Region',x_column]).drop(case4_x2.columns[0],axis=1)
print(case4_x2.head())
print('number of rows in dataset =',case4_x2.shape[0])
case4_x2.to_csv('../outputs/'+x_name+'_8760_2daytype_season_wkd.csv')

     Region  Season_Group  Weekday  Hour  Hour_Tot  Wind_Gen_Tot  Wind_Gen_Avg
0  ERC_PHDL             1    False     1        26       14093.0    542.038462
1  ERC_PHDL             1    False     2        26       14612.0    562.000000
2  ERC_PHDL             1    False     3        26       15178.0    583.769231
3  ERC_PHDL             1    False     4        26       15584.0    599.384615
4  ERC_PHDL             1    False     5        26       16131.0    620.423077
number of rows in dataset = 13036

     R_Group R_Subgroup  Season  Month  Day  Hour  TRG4       Date  DOW  \
5913     ERC       PHDL  summer      9  247    10  33.0 2016-09-03    5   
5914     ERC       PHDL  summer      9  247    11  49.0 2016-09-03    5   
4883     ERC       PHDL  summer      7  204    12  51.0 2016-07-22    4   
4880     ERC       PHDL  summer      7  204     9  53.0 2016-07-22    4   
5028     ERC       PHDL  summer      7  210    13  55.0 2016-07-28    3   

      Weekday  Season_Group  Hour_Tot  W

## 4 hour interval day types

In [7]:
#read in 4 hour interval counter
interval_4hr = pd.read_csv('inputs/sequential_hours.csv')
interval_4hr = interval_4hr.drop(columns=['2-hr','8-hr','12-hr','24-hr','48-hr','120-hr'])
print(interval_4hr)

#add an hour counter
x2['Hour_Counter'] = (x2['Hour']) + (x2['Day'] - 1) * 24
x2 = x2.sort_values(by=['Region','Hour_Counter'])
unique_hc = pd.Series(x2['Hour_Counter'].unique()).dropna()
#print(unique_hc.tail(2))

x3 = pd.merge(x2,interval_4hr,on='Hour_Counter',how='left')
print(x3)

      Hour_Counter  4-hr
0                1     1
1                2     1
2                3     1
3                4     1
4                5     2
...            ...   ...
8755          8756  2189
8756          8757  2190
8757          8758  2190
8758          8759  2190
8759          8760  2190

[8760 rows x 2 columns]
          Region R_Group R_Subgroup  Season  Month  Day  Hour   TRG4  \
0       ERC_PHDL     ERC       PHDL  winter      1    1     1  653.0   
1       ERC_PHDL     ERC       PHDL  winter      1    1     2  687.0   
2       ERC_PHDL     ERC       PHDL  winter      1    1     3  605.0   
3       ERC_PHDL     ERC       PHDL  winter      1    1     4  579.0   
4       ERC_PHDL     ERC       PHDL  winter      1    1     5  538.0   
...          ...     ...        ...     ...    ...  ...   ...    ...   
867235  WEC_SDGE     WEC       SDGE  winter     12  365    20   71.0   
867236  WEC_SDGE     WEC       SDGE  winter     12  365    21   69.0   
867237  WEC_SDGE     WEC   

## Case 5: Monthly, single day type, 4 hour intervals (72 segments)
#### Methodology: use groupby by month, 4 hour intervals

In [8]:
case5_x = x3.copy()

aggregations = {x_column:['count',sum,'mean']}
case5 = case5_x.groupby(['Region','Month','4-hr'],as_index=False).agg(aggregations)
case5.columns = case5.columns.droplevel(0)
case5.columns = ['Region','Month','4-hr','Hour_Tot',x_name2+'_Tot',x_name2+'_Avg']
print(case5.head())
print('number of rows in dataset =',case5.shape[0])
case5.to_csv('../outputs/'+x_name+'_segments_2daytype_month_4hr.csv')
print()

case5_x2 = pd.merge(case5_x,case5,on=['Region','Month','4-hr'],how='left')
case5_x2 = case5_x2.sort_values(['Region',x_column]).drop(case5_x2.columns[0],axis=1)
print(case5_x2.head())
print('number of rows in dataset =',case5_x2.shape[0])
case5_x2.to_csv('../outputs/'+x_name+'_8760_2daytype_month_4hr.csv')

     Region  Month  4-hr  Hour_Tot  Wind_Gen_Tot  Wind_Gen_Avg
0  ERC_PHDL      1     1         4        2524.0        631.00
1  ERC_PHDL      1     2         4        1702.0        425.50
2  ERC_PHDL      1     3         4        1133.0        283.25
3  ERC_PHDL      1     4         4         936.0        234.00
4  ERC_PHDL      1     5         4        1575.0        393.75
number of rows in dataset = 99658

     R_Group R_Subgroup  Season  Month  Day  Hour  TRG4       Date  DOW  \
5913     ERC       PHDL  summer      9  247    10  33.0 2016-09-03    5   
5914     ERC       PHDL  summer      9  247    11  49.0 2016-09-03    5   
4883     ERC       PHDL  summer      7  204    12  51.0 2016-07-22    4   
4880     ERC       PHDL  summer      7  204     9  53.0 2016-07-22    4   
5028     ERC       PHDL  summer      7  210    13  55.0 2016-07-28    3   

      Weekday  Hour_Counter  4-hr  Hour_Tot  Wind_Gen_Tot  Wind_Gen_Avg  
5913    False          5914  1479         4         228.0     

## Case 6: Bi-monthly weekend/weekday day-types, 4-hour intervals
#### Methodology: use groupby function and bimonthly groups

In [9]:
case6_x = x3.copy()
case6_bimonth = pd.read_csv('inputs/season_bimonthly.csv')
case6_bimonth = case6_bimonth.drop(['seasonal'], axis=1)
case6_bimonth = case6_bimonth.rename(columns={'bimonthly':'Bimonth','month':'Month'})

case6_x = pd.merge(case6_x, case6_bimonth, on='Month', how='left')

aggregations = {x_column:['count',sum,'mean']}
case6 = case6_x.groupby(['Region','Bimonth','4-hr'],as_index=False).agg(aggregations)
case6.columns = case6.columns.droplevel(0)
case6.columns = ['Region','Bimonth','4-hr','Hour_Tot',x_name2+'_Tot',x_name2+'_Avg']
print(case6.head())
print('number of rows in dataset =',case6.shape[0])
case6.to_csv('../outputs/'+x_name+'_segments_2daytype_bimonth_4hr.csv')
print()

case6_x2 = pd.merge(case6_x,case6,on=['Region','Bimonth','4-hr'],how='left')
case6_x2 = case6_x2.sort_values(['Region',x_column]).drop(case6_x2.columns[0],axis=1)
print(case6_x2.head())
print('number of rows in dataset =',case6_x2.shape[0])
case6_x2.to_csv('../outputs/'+x_name+'_8760_2daytype_bimonth_4hr.csv')

     Region  Bimonth  4-hr  Hour_Tot  Wind_Gen_Tot  Wind_Gen_Avg
0  ERC_PHDL        1     1         4        2524.0        631.00
1  ERC_PHDL        1     2         4        1702.0        425.50
2  ERC_PHDL        1     3         4        1133.0        283.25
3  ERC_PHDL        1     4         4         936.0        234.00
4  ERC_PHDL        1     5         4        1575.0        393.75
number of rows in dataset = 94143



MemoryError: Unable to allocate 52.9 MiB for an array with shape (8, 867240) and data type int64

## Case 7: Season-based months, weekend/weekday day-types, 4-hour intervals
#### Methodology: groupby function and applied season groups

In [ ]:
case7_x = x3.copy()
case7_seasons = pd.read_csv('inputs/season_bimonthly.csv')
case7_seasons = case7_seasons.drop(['bimonthly'], axis=1)
case7_seasons = case7_seasons.rename(columns={'seasonal':'Season_Group','month':'Month'})

case7_x = pd.merge(case7_x, case7_seasons, on='Month', how='left')
#print(case4_load)

aggregations = {x_column:['count',sum,'mean']}
case7 = case7_x.groupby(['Region','Season','Weekday','4-hr'],as_index=False).agg(aggregations)
case7.columns = case7.columns.droplevel(0)
case7.columns = ['Region','Season','Weekday','4-hr','Hour_Tot',x_name2+'_Tot',x_name2+'_Avg']
print(case7.head())
print('number of rows in dataset =',case7.shape[0])
case7.to_csv('../outputs/'+x_name+'_segments_2daytype_season_wkd_4hr.csv')
print()

case7_x2 = pd.merge(case7_x,case7,on=['Region','Season','Weekday','4-hr'],how='left')
case7_x2 = case7_x2.sort_values(['Region',x_column]).drop(case7_x2.columns[0],axis=1)
print(case7_x2.head())
print('number of rows in dataset =',case7_x2.shape[0])
case7_x2.to_csv('../outputs/'+x_name+'_8760_2daytype_season_wkd_4hr.csv')

# Day type: Three (Weekday, Weekend, Peak Load)
## Find Peak Load Days in Each Month

In [ ]:
#create temporary DF to find peak
test = x.copy()

#groupby region, month, and day to sum the total day
aggregations1 = {x_column:sum}
test_sum = test.groupby(['Region','Month','Day'],as_index=False).agg(aggregations1)
#test1.columns = test1.columns.droplevel(0)
test_sum.columns = ['Region','Month','Day',x_name2+'_Tot']
#print(test_sum.head())
#print('number of rows in dataset =',test_sum.shape[0])

test3 = pd.merge(test,test_sum,on=['Region','Month','Day'],how='left')
#print(test3)

#groupby region and month to find maximum 
aggregations2 = {x_name2+'_Tot':max}
test_max = test_sum.groupby(['Region','Month'],as_index=False).agg(aggregations2)
test_max.columns = ['Region','Month',x_name2+'_Max']
#print(test_max.head())

test4 = pd.merge(test3,test_max,on=['Region','Month'],how='left')
print(test4)

In [ ]:
x_peak = x2.copy()

x_peak = pd.merge(x_peak,test4,on=['Region','R_Group','R_Subgroup','Season','Month','Day','Hour',x_column],how='left')
x_peak = x_peak.rename(columns={'Weekday':'Day_Type'})

#Return True if the load total equals the day identified as the max
x_peak.loc[x_peak['Day_Type'] == True, 'Day_Type'] = 'Weekday'
x_peak.loc[x_peak['Day_Type'] == False, 'Day_Type'] = 'Weekend'
x_peak.loc[x_peak[x_name2+'_Tot'] == x_peak[x_name2+'_Max'], 'Day_Type'] = 'Peak'
x_peak = x_peak.drop([x_name2+'_Tot',x_name2+'_Max'], axis=1)
print(x_peak)

## Case 1: Monthly, Weekend/weekday/peak day-types, 24 hours (864 segments)
#### Methodology: similar to two day type, just adding in peak day types to sort by

In [ ]:
case1_x = x_peak.copy()

aggregations = {x_column:['count',sum,'mean']}
case1 = case1_x.groupby(['Region','Month','Day_Type','Hour'],as_index=False).agg(aggregations)
case1.columns = case1.columns.droplevel(0)
case1.columns = ['Region','Month','Day_Type','Hour','Hour_Tot',x_name2+'_Tot',x_name2+'_Avg']
print(case1.head())
print('number of rows in dataset =',case1.shape[0])
case1.to_csv('../outputs/'+x_name+'_segments_3daytype_monthly_24hr.csv')
print()

case1_x2 = pd.merge(case1_x,case1,on=['Region','Month','Day_Type','Hour'],how='left')
case1_x2 = case1_x2.sort_values(['Region',x_column])
print(case1_x2.head(3))
print('number of rows in dataset =',case1_x2.shape[0])
case1_x2.to_csv('../outputs/'+x_name+'_8760_3daytype_monthly_24hr.csv')

## Case 2: Season, weekend/weekday/peak day-types, 24-hours (216 segments)

In [ ]:
case2_x = x_peak.copy()

aggregations = {x_column:['count',sum,'mean']}
case2 = case2_x.groupby(['Region','Season','Day_Type','Hour'],as_index=False).agg(aggregations)
case2.columns = case2.columns.droplevel(0)
case2.columns = ['Region','Season','Day_Type','Hour','Hour_Tot',x_name2+'_Tot',x_name2+'_Avg']
print(case2.head())
print('number of rows in dataset =',case2.shape[0])
case2.to_csv('../outputs/'+x_name+'_segments_3daytype_season_24hr.csv')
print()

case2_x2 = pd.merge(case2_x,case2,on=['Region','Season','Day_Type','Hour'],how='left')
case2_x2 = case2_x2.sort_values(['Region',x_column])
print(case1_x2.head(3))
print('number of rows in dataset =',case1_x2.shape[0])
case2_x2.to_csv('../outputs/'+x_name+'_8760_3daytype_season_24hr.csv')

## Case 3: Annual, weekend/weekday/peak day-types, 24-hours (72 segments)

In [ ]:
case3_x = x_peak.copy()

aggregations = {x_column:['count',sum,'mean']}
case3 = case3_x.groupby(['Region','Day_Type','Hour'],as_index=False).agg(aggregations)
case3.columns = case3.columns.droplevel(0)
case3.columns = ['Region','Day_Type','Hour','Hour_Tot',x_name2+'_Tot',x_name2+'_Avg']
print(case3.head())
print('number of rows in dataset =',case3.shape[0])
case3.to_csv('../outputs/'+x_name+'_segments_3daytype_annual_24hr.csv')
print()

case3_x2 = pd.merge(case3_x,case3,on=['Region','Day_Type','Hour'],how='left')
case3_x2 = case3_x2.sort_values(['Region',x_column])
print(case3_x2.head(3))
print('number of rows in dataset =',case3_x2.shape[0])
case3_x2.to_csv('../outputs/'+x_name+'_8760_3daytype_annual_24hr.csv')

## 4 hour interval

In [ ]:
#read in 4 hour interval counter
interval_4hr = pd.read_csv('inputs/sequential_hours.csv')
interval_4hr = interval_4hr.drop(columns=['2-hr','8-hr','12-hr','24-hr','48-hr','120-hr'])
#print(interval_4hr)

#add an hour counter
x_peak['Hour_Counter'] = (x_peak['Hour']) + (x_peak['Day'] - 1) * 24
x_peak = x_peak.sort_values(by=['Region','Hour_Counter'])
unique_hc = pd.Series(x_peak['Hour_Counter'].unique()).dropna()
#print(unique_hc.tail(2))

x_peak2 = pd.merge(x_peak,interval_4hr,on='Hour_Counter',how='left')

## Case 4: Bi-monthly, weekend/weekday/peak day-types, 4-hour intervals

In [ ]:
case4_x = x_peak2.copy()
case4_bimonth = pd.read_csv('inputs/season_bimonthly.csv')
case4_bimonth = case4_bimonth.drop(['seasonal'], axis=1)
case4_bimonth = case4_bimonth.rename(columns={'bimonthly':'Bimonth','month':'Month'})

case4_x = pd.merge(case4_x, case4_bimonth, on='Month', how='left')

aggregations = {x_column:['count',sum,'mean']}
case4 = case4_x.groupby(['Region','Bimonth','Day_Type','4-hr'],as_index=False).agg(aggregations)
case4.columns = case4.columns.droplevel(0)
case4.columns = ['Region','Bimonth','Day_Type','4-hr','Hour_Tot',x_name2+'_Tot',x_name2+'_Avg']
print(case4.head())
print('number of rows in dataset =',case4.shape[0])
case4.to_csv('../outputs/'+x_name+'_segments_3daytype_bimonth_4hr.csv')
print()

case4_x2 = pd.merge(case4_x,case4,on=['Region','Bimonth','Day_Type','4-hr'],how='left')
case4_x2 = case4_x2.sort_values(['Region',x_column])
print(case4_x2.head())
print('number of rows in dataset =',case4_x2.shape[0])
case4_x2.to_csv('../outputs/'+x_name+'_8760_3daytype_bimonth_4hr.csv')

## Case 5: Season-based months, weekend/weekday/peak day-types, 4-hour intervals

In [ ]:
case5_x = x_peak2.copy()
case5_seasons = pd.read_csv('inputs/season_bimonthly.csv')
case5_seasons = case5_seasons.drop(['bimonthly'], axis=1)
case5_seasons = case5_seasons.rename(columns={'seasonal':'Season_Group','month':'Month'})

case5_x = pd.merge(case5_x, case5_seasons, on='Month', how='left')

aggregations = {x_column:['count',sum,'mean']}
case5 = case5_x.groupby(['Region','Season_Group','Day_Type','4-hr'],as_index=False).agg(aggregations)
case5.columns = case5.columns.droplevel(0)
case5.columns = ['Region','Season_Group','Day_Type','4-hr','Hour_Tot',x_name2+'_Tot',x_name2+'_Avg']
print(case5.head())
print('number of rows in dataset =',case5.shape[0])
case5.to_csv('../outputs/'+x_name+'_segments_3daytype_seasonbased_4hr.csv')
print()

case5_x2 = pd.merge(case5_x,case5,on=['Region','Season_Group','Day_Type','4-hr'],how='left')
case5_x2 = case5_x2.sort_values(['Region',x_column])
print(case5_x2.head())
print('number of rows in dataset =',case5_x2.shape[0])
case5_x2.to_csv('../outputs/'+x_name+'_8760_3daytype_seasonbased_4hr.csv')

## Case 6: Season, weekend/weekday/peak day-types, 4-hour intervals

In [ ]:
case6_x = x_peak2.copy()

aggregations = {x_column:['count',sum,'mean']}
case6 = case6_x.groupby(['Region','Season','Day_Type','4-hr'],as_index=False).agg(aggregations)
case6.columns = case6.columns.droplevel(0)
case6.columns = ['Region','Season','Day_Type','4-hr','Hour_Tot',x_name2+'_Tot',x_name2+'_Avg']
print(case6.head())
print('number of rows in dataset =',case6.shape[0])
case6.to_csv('../outputs/'+x_name+'_segments_3daytype_season_4hr.csv')
print()

case6_x2 = pd.merge(case6_x,case6,on=['Region','Season','Day_Type','4-hr'],how='left')
case6_x2 = case6_x2.sort_values(['Region',x_column])
print(case6_x2.head())
print('number of rows in dataset =',case6_x2.shape[0])
case6_x2.to_csv('../outputs/'+x_name+'_8760_3daytype_season_4hr.csv')